In [ ]:
import json
import time
import requests
import random
from tqdm import tqdm

In [ ]:
def get_url(time_0='2022-03-22', time_1='2022-03-23', pages=100):
    url_list = []
    for page in range(1, pages+1):
        url = f"https://reportapi.eastmoney.com/report/list?cb=datatable5008116&industryCode=*&pageSize=50&industry=*&rating=*&ratingChange=*&beginTime={time_0}&endTime={time_1}&pageNo={page}&fields=&qType=1&orgCode=&rcode=&p={page}&pageNum={page}&pageNumber={page}"
        url_list.append(url)
    return url_list

def get_report(url_list):
    query_list = []
    for i in tqdm(range(len(url_list))):
        url = url_list[i]
        res = requests.get(url)
        res_text = res.text
        res_text = res_text[17:-1]
        res_js = json.loads(res_text)
        if len(res_js['data'])==0:
            break
        query_list.extend(res_js['data'])
    return query_list

In [ ]:
url_list = get_url(time_0='2020-04-15', time_1='2022-04-15', pages=2000)

In [ ]:
query_list = get_report(url_list[0:1])

In [ ]:
query_list[0]

In [ ]:
from bs4 import BeautifulSoup

def get_content(text_link):

    html = requests.get(text_link).content
    soup = BeautifulSoup(html, "lxml")

    res = soup.find('div', class_='ctx-content')
    
    if res != None:
        content = [i.text for i in res.find_all('p')]
        content = ''.join(content)
        content = content.replace('\u3000\u3000', '')
        content = content.replace('\u3000\u3000', '')
        content = content.replace('\r', '')
        content = content.replace('\n', '')
        content = content.replace(' ', '')
        return content

    return None

query = query_list[0]

infoCode = query['infoCode']
content_url = f"https://data.eastmoney.com/report/zw_industry.jshtml?infocode={infoCode}"
content = get_content(content_url)
query['content_url'] = content_url
query['content'] = content

In [ ]:
query

In [ ]:
import numpy as np

data = np.load('files/eastmoney_ggyb.pkl', allow_pickle=True)

In [ ]:
all = []
names = []
for item in data:
    if item['stockName'] not in names:
        names.append(item['stockName'])
        all.append(
            {
                'stockName':item['stockName'],
                'stockCode':item['stockCode'],
                'indvInduCode':item['indvInduCode'],
                'indvInduName':item['indvInduName'],
            }
        )

In [ ]:
import pandas as pd

df = pd.DataFrame(all)

In [ ]:
df.head(10)

In [ ]:
tmp = list(set(df['indvInduName']))

In [ ]:
df['stockCode'] = df['stockCode'].astype(str)

In [ ]:
df.to_csv('./files/stock_list.csv', index=None)

In [8]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

df = pd.read_csv('./files/stock_list.csv')

In [4]:
len(df)

2490

In [100]:
# for i in range(len(df)):

stockCode = df.iloc[0]['stockCode']
# content_url = f'http://f10.eastmoney.com/f10_v2/OperationsRequired.aspx?code=sh{stockCode}'
content_url = f'http://f10.eastmoney.com/CoreConception/index?&code=SH{stockCode}'

html = requests.get(content_url).content

soup = BeautifulSoup(html, 'lxml')

res = soup.findAll(name='p', class_='p_div')


In [101]:
res

[]

In [104]:
df

,stockName,stockCode,indvInduCode,indvInduName
0,明阳智能,601615,1032.0,风电设备
1,帅丰电器,605336,456.0,家电行业
2,兆易创新,603986,1036.0,半导体
3,华纳药厂,688799,465.0,化学制药
4,英科再生,688087,454.0,塑料制品
...,...,...,...,...
2485,杰普特,688025,459.0,电子元件
2486,剑桥科技,603083,448.0,通信设备
2487,浙江富润,600070,447.0,电子信息
2488,众合科技,925,429.0,交运设备
